In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score

# from aijack.attack import GAN_Attack
from aijack.collaborative import FedAvgClient, FedAvgServer
from aijack.utils import NumpyDataset

# Number of channels in the training images. For color images this is 3
nc = 1
# Size of z latent vector (i.e. size of generator input)
nz = 100
# Size of feature maps in generator
ngf = 64
# Batch Size
batch_size = 64


class Generator(nn.Module):
    def __init__(self, nz, nc, ngf):
        super(Generator, self).__init__()
        # Generator Code (from https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html)
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(nz, ngf * 4, 4, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 3, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 2, ngf * 1, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 1),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 1, 1, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 5),
            nn.Tanh(),
            nn.MaxPool2d(3, 3, 1),
            nn.Conv2d(32, 64, 5),
            nn.Tanh(),
            nn.MaxPool2d(3, 3, 1),
        )

        self.lin = nn.Sequential(
            nn.Linear(256, 200), nn.Tanh(), nn.Linear(200, 11), nn.LogSoftmax()
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.reshape((-1, 256))
        x = self.lin(x)
        return x


def prepare_dataloaders():
    at_t_dataset_train = torchvision.datasets.MNIST(
        root="./", train=True, download=True
    )
    at_t_dataset_test = torchvision.datasets.MNIST(
        root="./", train=False, download=True
    )

    X = at_t_dataset_train.data.numpy()
    y = at_t_dataset_train.targets.numpy()

    # ToTensor：画像のグレースケール化（RGBの0~255を0~1の範囲に正規化）、Normalize：Z値化（RGBの平均と標準偏差を0.5で決め打ちして正規化）
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
    )

    # idx_1 = random.sample(range(400), 200)
    # idx_2 = list(set(range(400)) - set(idx_1))
    idx_1 = np.where(y < 5)[0]
    idx_2 = np.where(y >= 5)[0]

    global_trainset = NumpyDataset(
        at_t_dataset_test.data.numpy(),
        at_t_dataset_test.targets.numpy(),
        transform=transform,
    )
    global_trainloader = torch.utils.data.DataLoader(
        global_trainset, batch_size=batch_size, shuffle=True, num_workers=2
    )
    trainset_1 = NumpyDataset(X[idx_1], y[idx_1], transform=transform)
    trainloader_1 = torch.utils.data.DataLoader(
        trainset_1, batch_size=batch_size, shuffle=True, num_workers=2
    )
    trainset_2 = NumpyDataset(X[idx_2], y[idx_2], transform=transform)
    trainloader_2 = torch.utils.data.DataLoader(
        trainset_2, batch_size=batch_size, shuffle=True, num_workers=2
    )

    return X, y, [trainloader_1, trainloader_2], global_trainloader, [200, 200]


def main():
    device = torch.device("cuda:0") if torch.cuda.is_available() else "cpu"
    print(device)

    X, y, trainloaders, global_trainloader, dataset_nums = prepare_dataloaders()

    criterion = nn.CrossEntropyLoss()
    client_num = 2

    net_1 = Net()
    client_1 = FedAvgClient(net_1, user_id=0)
    client_1.to(device)
    optimizer_1 = optim.SGD(
        client_1.parameters(), lr=0.02, weight_decay=1e-7, momentum=0.9
    )

    net_2 = Net()
    client_2 = FedAvgClient(net_2, user_id=1)
    client_2.to(device)
    optimizer_2 = optim.SGD(
        client_2.parameters(), lr=0.02, weight_decay=1e-7, momentum=0.9
    )

    clients = [client_1, client_2]
    optimizers = [optimizer_1, optimizer_2]

    global_model = Net()
    global_model.to(device)
    server = FedAvgServer(clients, global_model)

    for epoch in range(5):
        for client_idx in range(client_num):
            client = clients[client_idx]
            trainloader = trainloaders[client_idx]
            optimizer = optimizers[client_idx]

            running_loss = 0.0
            for _, data in enumerate(trainloader, 0):
                # get the inputs; data is a list of [inputs, labels]
                inputs, labels = data
                inputs = inputs.to(device)

                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward + backward + optimize
                outputs = client(inputs)
                loss = criterion(outputs, labels.to(torch.int64))
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

            print(
                f"epoch {epoch}: client-{client_idx+1}",
                running_loss / dataset_nums[client_idx],
            )
        server.receive()
        server.update()
        server.distribtue()

        in_preds = []
        in_label = []
        with torch.no_grad():
            for data in global_trainloader:
                inputs, labels = data
                inputs = inputs.to(device)
                outputs = server.server_model(inputs)
                in_preds.append(outputs)
                in_label.append(labels)
            in_preds = torch.cat(in_preds)
            in_label = torch.cat(in_label)
        print(
            f"epoch {epoch}: accuracy is ",
            accuracy_score(
                np.array(torch.argmax(in_preds, axis=1).cpu()), np.array(in_label)
            ),
        )



if __name__ == "__main__":
    main()

cuda:0
True
True
True
True
True
True
True
True
True
True


d:\Projects\FederatedLearningTensorflow\.venv\lib\site-packages\torch\nn\modules\container.py:141: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


KeyboardInterrupt: 